In [9]:
import pandas as pd
import json

In [3]:
import sys
sys.path.append("..")
from translate import translate_text

## Hotpot

In [7]:
df_dev_hp = pd.read_json("data/hotpot_dev_v1_simplified_multilingual.json")
df_dev_hp.head()

,question,answer,type,type_es_gt,question_es_gt,answer_es_gt,type_pt_gt,question_pt_gt,answer_pt_gt,type_fr_gt,question_fr_gt,answer_fr_gt,type_it_gt,question_it_gt,answer_it_gt
0,Were Scott Derrickson and Ed Wood of the same ...,yes,comparison,comparación,¿Scott Derrickson y Ed Wood eran de la misma n...,Sí,comparação,Scott Derrickson e Ed Wood eram da mesma nacio...,sim,comparaison,Scott Derrickson et Ed Wood étaient-ils de la ...,Oui,confronto,Scott Derrickson e Ed Wood erano della stessa ...,SÌ
1,What government position was held by the woman...,Chief of Protocol,bridge,puente,¿Qué puesto en el gobierno ocupaba la mujer qu...,Jefe de Protocolo,ponte,Que posição governamental foi ocupada pela mul...,chefe de protocolo,pont,Quel poste gouvernemental occupait la femme qu...,Chef de protocole,ponte,Quale posizione di governo era ricoperta dalla...,Capo del protocollo
2,"What science fantasy young adult series, told ...",Animorphs,bridge,puente,¿Qué serie de fantasía científica para adultos...,animorfos,ponte,Que série de fantasia científica para jovens a...,Animorfos,pont,Quelle série de science-fiction pour jeunes ad...,Animorphes,ponte,Quale serie di fantascienza per giovani adulti...,Animorfi
3,Are the Laleli Mosque and Esma Sultan Mansion ...,no,comparison,comparación,¿La Mezquita Laleli y la Mansión Esma Sultan e...,No,comparação,A Mesquita Laleli e a Mansão Esma Sultan estão...,não,comparaison,La mosquée Laleli et le manoir Esma Sultan son...,Non,confronto,La Moschea Laleli e la Esma Sultan Mansion si ...,NO
4,"The director of the romantic comedy ""Big Stone...","Greenwich Village, New York City",bridge,puente,¿En qué ciudad de Nueva York reside el directo...,"Greenwich Village, Nueva York",ponte,"O diretor da comédia romântica ""Big Stone Gap""...","Greenwich Village, Nova York",pont,"Le réalisateur de la comédie romantique ""Big S...","Greenwich Village, New York",ponte,"Il regista della commedia romantica ""Big Stone...","Greenwich Village, New York City"


In [8]:
df_dev_hp.info()


<class 'pandas.core.frame.DataFrame'>
Index: 7405 entries, 0 to 7404
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question        7405 non-null   object
 1   answer          7405 non-null   object
 2   type            7405 non-null   object
 3   type_es_gt      7405 non-null   object
 4   question_es_gt  7405 non-null   object
 5   answer_es_gt    7405 non-null   object
 6   type_pt_gt      7405 non-null   object
 7   question_pt_gt  7405 non-null   object
 8   answer_pt_gt    7405 non-null   object
 9   type_fr_gt      7405 non-null   object
 10  question_fr_gt  7405 non-null   object
 11  answer_fr_gt    7405 non-null   object
 12  type_it_gt      7405 non-null   object
 13  question_it_gt  7405 non-null   object
 14  answer_it_gt    7405 non-null   object
dtypes: object(15)
memory usage: 925.6+ KB


In [5]:
def run_translate_dataset(
        df_dev, 
        lang_cod='es',  
        cols=['type', 'question', 'answer'], 
        output_file="hotpot_dev_v1_simplified_multilingual.json",
        overwrite = False
        ):
    
    for col_name in cols:
        new_col_name = col_name+'_'+lang_cod+'_gt'
        if new_col_name in df_dev.columns and not overwrite:
            continue

        list_translations = list()
        cached_translations = dict()
        print(f"{col_name} {lang_cod}")
        i = 0
        for text in list(df_dev[col_name]):
            if text in cached_translations:
                list_translations.append(cached_translations[text])
            else:
                result = translate_text(target=lang_cod, text=text)['translatedText']
                list_translations.append(result)
                cached_translations[text] = result
            i += 1
            if i % 1000 == 0:
                print(i)

        df_dev[new_col_name] = list_translations
        df_dev.to_json(f"data/{output_file}")


In [6]:
cols=['type', 'question', 'answer']
output_file="hotpot_dev_v1_simplified_multilingual.json"
languages = ['es', 'pt', 'fr', 'it']

for lang in languages:
    run_translate_dataset(df_dev_hp, lang, cols, output_file)

answer it
1000
2000
3000
4000
5000
6000
7000


In [11]:
df_dev_hp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7405 entries, 0 to 7404
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  7405 non-null   object
 1   answer    7405 non-null   object
 2   type      7405 non-null   object
dtypes: object(3)
memory usage: 173.7+ KB


# Fever

In [10]:
data_path = f"./data/fever_paper_dev.jsonl"
with open(data_path, "r") as json_file:
    json_list = list(json_file)

labels = []
claims = []

for json_str in json_list:
    json_str = json.loads(json_str)
    label = json_str["label"]
    claim = json_str["claim"]
    labels.append(label)
    claims.append(claim)

df_dev_fever = pd.DataFrame({'label': labels, 'claim': claims})
df_dev_fever.head()


,label,claim
0,NOT ENOUGH INFO,Colin Kaepernick became a starting quarterback...
1,NOT ENOUGH INFO,Tilda Swinton is a vegan.
2,SUPPORTS,Fox 2000 Pictures released the film Soul Food.
3,NOT ENOUGH INFO,Anne Rice was born in New Jersey.
4,REFUTES,Telemundo is a English-language television net...


In [11]:
cols=['label', 'claim']
output_file="fever_paper_dev_multilingual.json"
languages = ['es', 'pt', 'fr', 'it']

for lang in languages:
    run_translate_dataset(df_dev_fever, lang, cols, output_file)

label es
1000
2000
3000
4000
5000
6000
7000
8000
9000
claim es
1000
2000
3000
4000
5000
6000
7000
8000
9000
label pt
1000
2000
3000
4000
5000
6000
7000
8000
9000
claim pt
1000
2000
3000
4000
5000
6000
7000
8000
9000
label fr
1000
2000
3000
4000
5000
6000
7000
8000
9000
claim fr
1000
2000
3000
4000
5000
6000
7000
8000
9000
label it
1000
2000
3000
4000
5000
6000
7000
8000
9000
claim it
1000
2000
3000
4000
5000
6000
7000
8000
9000
